In [1]:
from flask import Flask, request, render_template
import json
import bson
import datetime
import requests
from locale import setlocale, LC_ALL
from pymongo import MongoClient
from bson.json_util import dumps

In [2]:
app = Flask(__name__)

In [3]:
#- Connexion au mongoDB
try:
    #- Récupérer une instance de la connexion au mongoDB
    connection = MongoClient("mongodb://localhost:27017")
    #- Récupérer le nom de notre base de données
    database = connection['sentiments']
    #- Récupérer le nom de la collection
    collection_comments = database['comments']
except:
    print("[Error]: Failed to connect to database..")
    connection = None

In [4]:
#- Fonctions
def get_comment():
    collec_agreg = collection_comments.aggregate([
        { "$sample": { "size": 1 } },
        { "$limit": 1 }
    ])
    random_comment = collec_agreg.next()
    return random_comment

class JsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime.datetime):
            return obj.isoformat()                # Format dates as ISO strings
        elif isinstance(obj, bson.Binary) and obj.subtype == bson.binary.UUID_SUBTYPE:
            return obj.as_uuid()                  # Format binary data as UUIDs
        elif hasattr(obj, '__str__'):
            return str(obj)                       # This will handle ObjectIds

        return super(JsonEncoder, self).default(obj)

#- Flask routing
@app.route("/api")
def index():
    #- Récupérer un commentaire au hasard
    comment_hespress = get_comment()
    #- Return the comment as json result
    return json.dumps(comment_hespress, cls = JsonEncoder, indent = 4, ensure_ascii = False)

In [ ]:
if __name__ == "__main__":
    app.run()